Notebooks created by Oliver Hausdörfer, Jonathan Külz, Hannah Markgraf.

**How to use this notebook:** You can upload this .ipynb-file to google colab for editing and running (go to [colab.research.google.com](colab.research.google.com) -> File -> Open Notebook -> Upload).

**How to submit this notebook:** After finishing the HandsOn on google colab, share the uploaded notebook (click share -> Anyone with link) and send the link with the <font color='red'>completed notebook and cell outputs</font> to the course supervisor oliver.hausdoerfer@tum.de. Include your name and student ID in this e-mail.

<font color='red'>Additionally:</font> this course is offered for the first time. Please provide us feedback regarding the programming exercises. If you want, you can use an anonymized e-mail to send us the feedback. If you use your TUM-Email the feedback will not be anonymous. Your feedback will not influence your evaluation results. Providing feedback is not required, but optional.

**All tasks that need to be completed by you are marked with:** "👉 TODO: ..."

# HandsOn 3: Q-Learning 🤖

At the end of the notebook, you will:

- Be able to implement a simple a Q-Learning agent.




## Q-Learning

**Goal: learning the Q-Function.**

- The **Q-Function** is the state-action-value function, i.e., it holds the values for taking each possible action in each state. For real world continuous problems, the Q-Function cannot be represented by a table. Instead, a function approximator, i.e., neural network, is chosen. The original Deep Q-Learning (DQN) approach was proposed in [this](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf) paper.

- In this notebook, we will stick to a small discrete toy problem, the [Frozen Lake Grid World](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) in which the Q-Function can be represented by a table.

- At the beginning, the Q-Table (or Neural Network) is initialized with arbitrary values for each state-action pair. As we explore the environment and update our Q-Table, i.e., during learning, it will give us better and better approximations of the true Q-Value.

- In our case, when the training is done, we have an optimal Q-Table. Consequently, we also have the optimal policy, since we can query the best action for every state.

The following is one Q-Learning algorithm taken from the Sutton & Barto book.


![q_learning_algorithm](https://raw.githubusercontent.com/OliEfr/HandsOnDRL_LabCourse/main/q_learning_algo_2.png)



### Implementation

In [ ]:
!pip install gymnasium

import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

from tqdm.notebook import tqdm

👉TODO: As always before starting have a look at the [FrozenLake environment](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).



We are going to use the environment with the following configuration:
- `map_name="4x4"`: a 4x4 grid version
- `is_slippery=False`: The agent always moves in the intended direction (deterministic).

The goal is to navigate from the starting state (S) to the goal state (G) by walking only on frozen tiles (F) and avoid holes (H).

In [ ]:
# 👉TODO: Create the FrozenLake-v1 environment using 4x4 map and non-slippery version (1 LOC)
env = 

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

We see with `Observation Space Shape Discrete(16)` that the observation is an integer representing the **agent’s current position as current_row * ncols + current_col (where both the row and col start at 0)**.

For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15. The number of possible observations is dependent on the size of the map. **For example, the 4x4 map has 16 possible observations.**


For instance, this is what state = 0 looks like:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/frozenlake.png" alt="FrozenLake">

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

The action space (the set of possible actions the agent can take) is discrete with 4 actions available 🎮:
- 0: GO LEFT
- 1: GO DOWN
- 2: GO RIGHT
- 3: GO UP

Reward function:
- Reach goal: +1
- Reach hole: 0
- Reach frozen: 0

## Create and Initialize the Q-table (Step 1)



![q_learning_algorithm](https://raw.githubusercontent.com/OliEfr/HandsOnDRL_LabCourse/main/q_learning_algo_2.png)

To init our Q-Table, we need to know how many rows (states) and columns (actions) to use. We already know their values from before, but we'll want to obtain them programmatically so that our algorithm generalizes for different environments. Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`


In [ ]:
# 👉 TODO: Complete this code cell
# States are often used as a synonym for observations, although they are technically not the same. See OpenAI Spinning Up documentation for details.
state_space = # number of states
print("There are ", state_space, " possible state.")

action_space = # number of observations
print("There are ", action_space, " possible actions.")

In [ ]:
# Create the Q-table of size (state_space, action_space) and initialized each values at 0 using np.zeros.
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

## Define the greedy policy

Since we are implementing an off-policy algorithm, we have two policies. This means we're using a **different policy for acting and updating the value function**.

- Epsilon-greedy policy (acting policy)
- Greedy-policy (updating policy)

The greedy policy will also be the final policy we'll have when the Q-learning agent completes training. The greedy policy is used to select an action using the Q-table.

The following slide is taking from the huggingface tutorial of DRL. Note that on- an off-policy DRL algorithms have their distinct advantages and disadvantages. More details regarding On-Policy (SARSA) and Off-Policy (Q-Learning) can be found [here](https://stackoverflow.com/questions/6848828/what-is-the-difference-between-q-learning-and-sarsa).
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-4.jpg" alt="Q-Learning" width="100%"/>


In [ ]:
# 👉TODO: Implement the greedy policy by selecting the action with the highest state-action value given a state (requires 1 LOC)
# and then return that action (requires another 1 LOC)
def greedy_policy(Qtable, state):



## Define the epsilon-greedy policy

Epsilon-greedy is the training policy that handles the exploration/exploitation trade-off.

The idea with epsilon-greedy:

- With *probability 1 - ɛ* : we do **exploitation** (i.e. our agent selects the action with the highest state-action pair value).

- With *probability ɛ*: we do **exploration** (trying a random action).

As the training continues, we progressively **reduce the epsilon value since we will need less and less exploration and more exploitation.**

In [ ]:
# 👉 TODO: complete the following function
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Randomly generate a number between 0 and 1
  random_num = random.uniform(0,1)
  # if random_num > greater than epsilon: exploitation
  if random_num > epsilon:
    # Take the action with the highest value given a state
    # (=greedy) (1 LOC)
    action = 
  # else: exploration
  else:
    # Take a random action
    action = 
  return action

## Define the hyperparameters ⚙️

The exploration related hyperparamters are some of the most important ones.

- We need to make sure that our agent **explores enough of the state space** to learn a good value approximation. To do that, we need to have progressive decay of the epsilon.
- If you decrease epsilon too fast (too high decay_rate), **you take the risk that your agent will be stuck**, since your agent didn't explore enough of the state space and hence can't solve the problem.

In [ ]:
# Training parameters
n_training_episodes = 10000  # Total training episodes
learning_rate = 0.7          # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate
eval_seed = []               # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05            # Minimum exploration probability
decay_rate = 0.0005            # Exponential decay rate for exploration prob

## Create the training loop method

The training loop goes like this:

```
Loop for each episode:
  Reduce epsilon
  Reset environment
    For step in max_steps:    
      Choose the action A_t using epsilon greedy policy
      Take the action A and observe the outcome state (s') and reward (r)
      Update the Q-value Q(s,a) using Bellman equation Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      If done, finish the episode
      Our next state is the new state
```

The most important thing to note here is the [Bellman Equation](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html), which is used to iteratively optimize the Q-Function. The Bellman Equation is a central element of many DRL algorithms.

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # Reduce epsilon
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Reset the environment
    state, info = env.reset()
    step = 0
    terminated = False
    truncated = False

    for step in range(max_steps):
      # 👉TODO: Choose the action At using epsilon greedy policy (1 LOC)
      action = 

      # 👉TODO: Take action At and observe Rt+1 and St+1 (1 LOC)
      new_state, reward, terminated, truncated, info = 

      # 👉TODO: Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)] (1 LOC)
      Qtable[state][action] = 

      # If terminated or truncated finish the episode
      if terminated or truncated:
        break

      # Our next state is the new state
      state = new_state
  return Qtable

## Train the Q-Learning agent 🏃

In [ ]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)

Inspect the Q-Table.

In [ ]:
Qtable_frozenlake

👉TODO: Will the learned policy lead us to the desired goal? Explain your decision _briefly_ with the help of one example from the Q-Table. Answer:

Evaluate the agent.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param max_steps: Maximum number of steps per episode
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param Q: The Q-table
  :param seed: The evaluation seed array (for taxi-v3)
  """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Take the action that has the maximum expected future reward given that state
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward

      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

Usually, you should have a mean reward of 1.0. The environment is relatively easy since the state space is small (16). What you can try to do is [to replace it with the slippery version](https://gymnasium.farama.org/environments/toy_text/frozen_lake/), which introduces stochasticity, making the environment more complex.

In [ ]:
# Evaluate the Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

You successfully implemented a Reinforcement Learning Agent using Q-Learning! The reason why such a simple algorithm can not be used for most real world problems is that it only applies for discrete environments where all Q-Values can be listed. Due to the curse of dimensionality it is not possible to store all Q-Values in systems with many states and actions (and in the continuous case anyway).

That is where neural networks and more advanced algorithms come into play. They tackle the problem of scaling DRL to continuous systems with larger spaces. Although the foundations of many of these algorithms are the same, they implement different methodologies to improve training routines and convergence. For instance, PPO limits the update step of the policy (_proximal_ policy optimization) and a experience replay buffer decorrelates samples for training (batch training as in DQN).

For problems in robotics you will usually use some of the more advanced algorithms. One example of such a task is handling a Rubik's Cube with a robot hand in [this paper](https://arxiv.org/pdf/1910.07113.pdf). 👉LAST TODO: _Briefly_ answer the following questions about that paper:

- What is domain randomization used for? Answer:

- Which DRL algorithm was used? Answer:

-  Do they use any type of behavioral cloning, if so, which one? Answer:

- What is the dimensionality of the observation space? Name three different observations chosen for the policy net! Answer:

# HandsOn 3: Finished 🤖

Here are some more resources and information to play around with

Guides on how to do DRL projects (recommended to check out before starting your practical!):


*   https://github.com/williamFalcon/DeepRLHacks
*   https://rockt.github.io/2018/08/29/msc-advice
*   https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html

Other resources
*   https://cs.stanford.edu/people/karpathy/reinforcejs/gridworld_td.html (recommended)
*   https://spinningup.openai.com/en/latest/spinningup/rl_intro.html
*   https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/
*   http://incompleteideas.net/book/the-book-2nd.html
*   DRL tutorials by hugging face: https://huggingface.co/learn/deep-rl-course/unit0/introduction



**How to submit this notebook:** After finishing the HandsOn on google colab, share the uploaded notebook (click share -> Anyone with link) and send the link with the <font color='red'>completed notebook and cell outputs</font> to the course supervisor oliver.hausdoerfer@tum.de. Include your name and student ID in this e-mail.

<font color='red'>Additionally:</font> this is a first years course. Please provide us feedback regarding the programming exercises. If you want, you can use an anonymized e-mail to send us the feedback. If you use your TUM-Email the feedback will not be anonymous. Your feedback will not influence your evaluation results. Providing feedback is not required, but optional.